In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [3]:
token = 'Tpk_c9732a8f250e4fd0b0de2305a1a9c91c'

In [4]:
df = pd.DataFrame(columns=['Ticker','Stock Price','Market Capitalization','# of shares to buy'])

In [6]:
def sublist(lst, x):
    for i in range(0,len(lst),x):
        yield lst[i:i+x]    

In [7]:
symbols_batch = list(sublist(stocks['Ticker'],100))

In [8]:
symbol_strings = []
for i in range(0,len(symbols_batch)):
    symbol_strings.append(','.join(symbols_batch[i]))

In [9]:
for i in symbol_strings:
    batch_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={i}&token={token}'
    data = requests.get(batch_call_url).json()
    for symbol in i.split(','):
        df = df.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],index=df.columns),
        ignore_index=True
        )
df

,Ticker,Stock Price,Market Capitalization,# of shares to buy
0,A,157.61,48194377747,N/A
1,AAL,21.90,14366827266,N/A
2,AAP,222.96,14278675088,N/A
3,AAPL,155.83,2601943993246,N/A
4,ABBV,121.34,210384003641,N/A
...,...,...,...,...
500,YUM,128.46,36594061000,N/A
501,ZBH,167.50,34953206159,N/A
502,ZBRA,557.29,29853279748,N/A
503,ZION,52.81,8513722355,N/A


In [10]:
portfolio_size = input('Enter the value of your portfolio:')
try:
    val = float(portfolio_size)
except ValueError:
    print('Please enter value as a number only')
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:10000000


In [11]:
position_size = val/len(df.index)
for i in range(0,len(df.index)):
    df.loc[i,'# of shares to buy'] = math.floor(position_size/df.loc[i,'Stock Price'])
df

,Ticker,Stock Price,Market Capitalization,# of shares to buy
0,A,157.61,48194377747,125
1,AAL,21.90,14366827266,904
2,AAP,222.96,14278675088,88
3,AAPL,155.83,2601943993246,127
4,ABBV,121.34,210384003641,163
...,...,...,...,...
500,YUM,128.46,36594061000,154
501,ZBH,167.50,34953206159,118
502,ZBRA,557.29,29853279748,35
503,ZION,52.81,8513722355,374


In [21]:
writer = pd.ExcelWriter('Recommended_trades.xlsx',engine='xlsxwriter')
df.to_excel(writer,'Recommended Trades',index=False)

In [22]:
background_color = '#4acded'
font_color = '#000000'
align = 'center'
header_frmt = writer.book.add_format({
    'font_color':font_color,
    'bg_color': '9b9c9c',
    'bold':'True',
    'align':align
})
str_frmt = writer.book.add_format({
    'font_color':font_color,
    'bg_color':background_color,
    'border':1,
    'align':align
})
dollar_frmt = writer.book.add_format({
    'num_format':'$0.00',
    'font_color':font_color,
    'bg_color':background_color,
    'border':1,
    'align':align
})
int_frmt = writer.book.add_format({
    'num_format':'0',
    'font_color':font_color,
    'bg_color':'#9ce148',
    'border':1,
    'align':align
})

In [23]:
column_formats = { 
                    'A': ['Ticker', str_frmt],
                    'B': ['Stock Price', dollar_frmt],
                    'C': ['Market Capitalization', dollar_frmt],
                    'D': ['# of shares to buy', int_frmt]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], header_frmt)

In [24]:
writer.save()